In [ ]:
%load_ext expectmagic

# 前処理（続き）

## ゴミの除去
### 正規表現（続き）

### 実習: Wikipediaのダンプの内部リンクをプレーンテキストに変換しよう

復習ともう少し複雑な正規表現の処理を兼ねて。

In [ ]:
# 実習用ファイルのダウンロード
from urllib import request

with open('data/uk.txt', 'wb') as f, request.urlopen('https://gist.githubusercontent.com/pizzacat83/9a9609d393c251b8df8e5bde96cd38c6/raw/141e1a9344e1cc7a8515e0ab85e57a92733435b7/uk.txt') as req:
    f.write(req.read())


In [ ]:
### tutor-only

import os
assert os.path.exists('data/uk.txt')

#### 1. ファイル読み込み（復習）
`data/uk.txt`を読み込んで，`text`という変数にファイルの内容を代入してください。その後，`text`を表示してください。

- ファイルを開く関数`open('ファイルパス', 'モード (optional)')`
- 一気にファイル全体を読みこむメソッド`f.read()`

In [ ]:
### answer

f = open('data/uk.txt')
text = f.read()
print(text)

`[[ほにゃらら]]`という形で内部リンクが表示されています。これを`ほにゃらら`に変換していきます。雑にやるなら`[[`と`]]`を全部削除してしまうわけですが，今回は対応している括弧だけを除去したいという設定でいきます。

#### 内部リンクの検出

**タスク**: `text`に含まれる`[[...]]`に，括弧ごとマッチするような正規表現`r`を作成してください。その後，その正規表現にマッチする文字列を全て列挙してください。

[Python公式チュートリアル](https://docs.python.org/ja/3/howto/regex.html)

- まず`re`をインポート
- 正規表現は`re.compile('source')`で作る
- 「どんな文字でもいい，何文字でもいい」: `.*`
  - 最短マッチにするなら`.*?`
- パターンを全て検索するには`r.findall('検索対象文字列')`


- `[tsg]`: `t`, `s`, `g`のいずれか1文字にマッチ
- `.`: 改行以外の任意の1文字にマッチ
- `*`: 0回以上の繰り返し
- `+`: 1回以上の繰り返し
- `?`: `*` `+`の後につけると最左マッチになる（デフォルトは最長マッチ）
- `pattern1|pattern2`: `pattern1`または`pattern2`にマッチ
- `(pattern)`: グルーピング。演算子の優先度を変えるのにも使えるし，`()`内にマッチしたテキストを得ることもできる
- `\`: 特殊文字のエスケープ

In [ ]:
%%expect ['[[イングランド]]', '[[スコットランド]]', '[[ウェールズ]]', '[[北アイルランド]]', '[[ロンドン]]', '[[エディンバラ]]', '[[カーディフ]]', '[[ベルファスト]]']
### answer

import re

#r = re.compile('\\[\\[.+?\\]\\]')
r = re.compile('\[\[.+?\]\]')

r.findall(text)

**注意点**
- `[`と`]`は`[tsg]`で使う特殊文字なので，文字としての`[`を指定したいときは`/`でエスケープ
- `'\['`ではpythonが`\`をpythonの文字列のエスケープ文字`\`として認識してしまう
  + `\n`と表示される文字列を作ろうとして`'\n'`とすると改行になってしまうので，`'\\n'`と書くのと同じ
  + つまり`'\\['`と書けば良い
  + 正規表現ではこうしたダブルバックスラッシュを多用するので，代わりに`\`を文字の`\`として扱ってくれる文字列リテラル`r'...'`を使うことも

#### 括弧の除去
前回は`《ルビ》`を丸ごと除去する（空文字列で置換する）という処理をしましたが，今回は丸ごと削除ではなく，括弧の中のテキストは残す必要があります。つまり，`[[ほにゃらら]]`を`ほにゃらら`で置換できれば良さそうです。

これを可能にするのが**グルーピング**です。検出した部分のうち括弧の中身だけを残したいので，括弧の中身をグルーピングします。グルーピングをするには，パターンを`()`で括ります。括った上で，まずは`findall`してみましょう。

In [ ]:
#%%expect ['イングランド', 'スコットランド', 'ウェールズ', '北アイルランド', 'ロンドン', 'エディンバラ', 'カーディフ', 'ベルファスト']
### answer

r = re.compile('\[\[(.+?)\]\]')

r.findall(text)

`[[ほにゃらら]]`を`ほにゃらら`に置換するのにも前回同様`r.sub('置換後', '対象文字列')`を使います。ただし，置換後に指定したいものはかっこの中身であり，マッチごとに異なります。置換後の文字列には先ほど作ったグループを指定することができます。かっこの中身は1番目のグループなので，`\1`と指定することで括弧の中身に置換することができます。

In [ ]:
# 例

r.sub('\\1', 'abc[[def]]g[[h]]i')

In [ ]:
### tutor-only

assert _ == 'abcdefghi'

**タスク**: `text`の`[[ほにゃらら]]`を`ほにゃらら`に置換してください。

In [ ]:
%%expect 'イギリスは四つの非独立国であるイングランド、スコットランド、ウェールズ、北アイルランドより構成される。それぞれの国は首都を持ち、ロンドン（イングランド）、エディンバラ（スコットランド）、カーディフ（ウェールズ）、ベルファスト（北アイルランド）がそれである。中でもイングランドの首都であるロンドンは、イギリスの首都としての機能も置かれる。'
### answer

r.sub('\\1', text)


## 正規化
本質的に区別する必要がない表層的な違いを統一します。

- アルファベットを全て小文字に
- 半角・全角の統一

ex. [たほいやの正規化関数](https://github.com/tsg-ut/slackbot/blob/513fcced5c957f6484f9e306e9b5586691eaa044/tahoiya/lib.js#L117-L129)

In [ ]:
# アルファベットの大文字・小文字変換

text = 'pizzacat83 lives in Japan.'

print(text)
print(text.lower()) # 小文字
print(text.upper()) # 大文字
print(text.capitalize()) # 1文字目だけ大文字
print(text.title()) # 単語頭だけ大文字

In [ ]:
# 半角・全角の統一

import jaconv

text = 'ぬるぽｶﾞｯＮｕｌｌPointer１２３４５67890'

print(text)
print(jaconv.h2z(text)) # Hankaku2Zenkaku デフォルトではカタカナのみ
print(jaconv.h2z(text, ascii=True, digit=True)) # 英数字も変換対象に
print(jaconv.z2h(text, ascii=True, digit=True)) # 全角→半角
print(jaconv.hira2kata(text)) # ひらがな→カタカナ